# Notebook 1 - Fondamentaux Python pour l'IA
# Analyse de données météorologiques en temps réel

🎯 Objectifs pédagogiques

Maîtriser les structures de données Python essentielles
Consommer des APIs REST avec requests
Manipuler des données JSON et CSV
Créer des visualisations basiques
Appliquer des statistiques descriptives

🌤️ Contexte du projet

Vous travaillez pour une startup AgTech qui développe des solutions d'agriculture intelligente. Votre mission : analyser les données météorologiques de plusieurs villes européennes pour optimiser les recommandations de plantation.

Partie 1 : Connexion aux APIs météo

🔧 Installation des bibliothèques


 À exécuter dans votre terminal ou cellule
# pip install requests pandas matplotlib seaborn numpy

In [44]:
#📥 Import et configuration
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta, date
import json
from dotenv import load_dotenv
import os

load_dotenv()

True

🌍 API OpenWeatherMap (gratuite)

Inscription : Créez un compte sur openweathermap.org


Clé API : Récupérez votre clé gratuite (40 000 appels/mois)



In [ ]:
### 💡 Première requête guidée
API_KEY_OWM = os.getenv('API_KEY_OWM')

def get_city_coordinates(city: str) -> dict:
    '''
    Récupère les coordonnées (latitude et longitude) d'une ville donnée en utilisant l'API OpenWeatherMap.

    args :
        city (str) : nom de la ville à rechercher.

    returns :
        dict : dictionnaire contenant les informations de la ville dont la latitude ('lat') et la longitude ('lon').
        None : si la ville n'est pas trouvée ou en cas d'erreur.
    '''
    url = 'http://api.openweathermap.org/geo/1.0/direct'
    params = {
        'q': city,
        'appid': API_KEY_OWM,
        'limit': 1
    }
    try:
        response = requests.get(url, params=params)

        # gérer l'erreur 401
        if response.status_code == 401:
            print('401 Unauthorized: Invalid API key')
            return None

        data_raw: dict = response.json()[0]
        return data_raw
    
    # gérer le cas où une ville n'est pas valide
    except IndexError:
        print(f'\'{city}\' is not a valid city')
    except Exception as e:
        print(f'error: {e}')

def get_weather_data(city: str) -> dict:
    """
    Récupère les données météo actuelles pour une ville

    Étapes à compléter :
    1. Construire l'URL avec les paramètres
    2. Faire l'appel API avec requests.get()
    3. Vérifier le status code
    4. Retourner les données JSON
    """
    try:
        city_data = get_city_coordinates(city)

        if city_data == None:
            print(f'Error: Couldn\'t get city data for \'{city}\'')
            return None

        latitude = city_data['lat']
        longitude = city_data['lon']

        url = 'https://api.openweathermap.org/data/2.5/weather'
        params = {
            'lat': latitude,
            'lon': longitude,
            'appid': API_KEY_OWM,
            'units': 'metric',
            'lang': 'fr'
        }

        response = requests.get(url, params=params)
        print(f'status code: {response.status_code}')
        return response.json()

    except Exception as e:
        print(f'error: {e}')

weather_in_montpellier = get_weather_data('montpellier')
print(weather_in_montpellier)

status code: 200
{'coord': {'lon': 3.8777, 'lat': 43.6085}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'ciel dégagé', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 26.95, 'feels_like': 26.64, 'temp_min': 26.95, 'temp_max': 27.71, 'pressure': 1014, 'humidity': 36, 'sea_level': 1014, 'grnd_level': 1005}, 'visibility': 10000, 'wind': {'speed': 11.32, 'deg': 350}, 'clouds': {'all': 0}, 'dt': 1753707900, 'sys': {'type': 2, 'id': 2038454, 'country': 'FR', 'sunrise': 1753676963, 'sunset': 1753729941}, 'timezone': 7200, 'id': 2992166, 'name': 'Montpellier', 'cod': 200}


In [ ]:
# Villes à analyser (agriculture européenne)
CITIES = ["Paris", "Berlin", "Madrid", "Roma, Lazio", "Amsterdam", "Vienna"] # Changement de 'Rome' pour 'Roma, Lazio' par nécessité de précision pour une API

for city in CITIES:
    weather = get_weather_data(city)
    print(f'weather in {city}: {weather}\n')

status code: 200
weather in Paris: {'coord': {'lon': 2.32, 'lat': 48.8589}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'ciel dégagé', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 18.68, 'feels_like': 18.69, 'temp_min': 17.86, 'temp_max': 20.75, 'pressure': 1021, 'humidity': 80, 'sea_level': 1021, 'grnd_level': 1011}, 'visibility': 10000, 'wind': {'speed': 3.09, 'deg': 260}, 'clouds': {'all': 0}, 'dt': 1753775348, 'sys': {'type': 1, 'id': 6550, 'country': 'FR', 'sunrise': 1753762803, 'sunset': 1753817647}, 'timezone': 7200, 'id': 6545270, 'name': 'Quartier du Palais-Royal', 'cod': 200}

status code: 200
weather in Berlin: {'coord': {'lon': 13.3889, 'lat': 52.517}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'nuageux', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 18.53, 'feels_like': 18.4, 'temp_min': 17.83, 'temp_max': 19.94, 'pressure': 1013, 'humidity': 75, 'sea_level': 1013, 'grnd_level': 1008}, 'visibility': 10000, 'wind': {'speed': 6.26, 

**Questions de débogage :**
- Que faire si l'API retourne une erreur 401 ?
- Comment gérer une ville introuvable ?

---

## Partie 2 : API complémentaire - Données historiques

### 📊 API Visual Crossing Weather (gratuite)
Alternative avec 1000 appels/jour gratuits : [visualcrossing.com](https://www.visualcrossing.com/weather-api)

In [75]:
# Configuration Visual Crossing
VC_API_KEY = os.getenv('VC_API_KEY')
VC_BASE_URL = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"

def get_historical_weather(city, start_date, end_date):
    """
    Récupère les données météo historiques

    Défis à résoudre :
    1. Construire l'URL avec les dates
    2. Gérer la pagination si nécessaire
    3. Extraire les données pertinentes du JSON complexe
    4. Convertir en DataFrame pandas
    """
    try:
        url = f"{VC_BASE_URL}/{city}/{start_date}/{end_date}"
        params = {
            'key': VC_API_KEY,
            'include': 'days',
            'elements': 'datetime,temp,humidity,precip,windspeed,pressure'
        }

        response  = requests.get(url, params)
        # print(response.status_code)

        # gérer l'erreur 400
        if response.status_code == 400:
            print(f'400 Bad Request: \'{city}\' is not a valid city')
            return None

        # gérer l'erreur 401
        if response.status_code == 401:
            print('401 Unauthorized: Invalid API key')
            return None

        data = response.json()

        weather_list = [
            {
                'date': day.get('datetime'),
                'ville': data.get('resolvedAddress'),
                'temperature': day.get('temp'),
                "humidite": day.get('humidity'),
                "precipitation": day.get('precip'),
                "vent": day.get('windspeed'),
                # "pressure": day.get('pressure')
            }
            for day in data.get('days')
            ]
        # print(weather_list)

        df_weather = pd.DataFrame(weather_list)
        df_weather.head()

        return df_weather
    
    except Exception as e:
        print(f'error: {e}')

get_historical_weather('montpellier', '2000-10-16', '2000-10-17')

,date,ville,temperature,humidite,precipitation,vent
0,2000-10-16,"Montpellier, Occitanie, France",56.0,83.6,0.034,9.4
1,2000-10-17,"Montpellier, Occitanie, France",60.6,68.9,0.000,13.8


### 🎯 Mission pratique
Récupérez les données des 7 (et non 30) derniers jours pour toutes vos villes et créez un DataFrame consolidé.

**Structure attendue :**
```
| date       | ville     | temperature | humidite | precipitation | vent |
|------------|-----------|-------------|----------|---------------|------|
| 2024-01-01 | Paris     | 12.5        | 75       | 2.3          | 15   |
```

---


In [79]:
list_cities_weather = []

today = date.today()
seven_days_ago = date.today() - timedelta(7)

for i in CITIES:
    city_info = get_historical_weather(i, seven_days_ago, today)
    list_cities_weather.append(city_info)

df_cities_weather = pd.concat(list_cities_weather)

df_cities_weather.head(80)

,date,ville,temperature,humidite,precipitation,vent
0,2025-07-22,"Paris, Île-de-France, France",66.6,71.0,0.002,10.4
1,2025-07-23,"Paris, Île-de-France, France",66.6,74.3,0.079,8.4
2,2025-07-24,"Paris, Île-de-France, France",65.2,83.8,0.489,9.6
3,2025-07-25,"Paris, Île-de-France, France",69.3,72.8,0.000,7.6
4,2025-07-26,"Paris, Île-de-France, France",70.6,67.6,0.000,10.0
5,2025-07-27,"Paris, Île-de-France, France",65.9,78.8,0.221,14.4
6,2025-07-28,"Paris, Île-de-France, France",66.0,65.9,0.000,8.6
7,2025-07-29,"Paris, Île-de-France, France",63.8,73.1,0.020,7.4
0,2025-07-22,"Berlin, Deutschland",63.5,86.8,0.256,14.4
1,2025-07-23,"Berlin, Deutschland",65.1,83.8,1.090,15.5


In [49]:
## Partie 3 : API supplémentaire - Qualité de l'air

### 🌬️ API OpenWeatherMap Air Pollution
def get_air_quality(lat, lon, api_key):
    """
    Récupère les données de qualité de l'air

    URL : http://api.openweathermap.org/data/2.5/air_pollution

    Étapes :
    1. Utiliser les coordonnées lat/lon des villes
    2. Récupérer l'indice AQI et les composants (PM2.5, PM10, O3, etc.)
    3. Joindre ces données avec vos données météo
    """
    pass

In [ ]:
### 🗺️ Obtenir les coordonnées
# def get_city_coordinates(city, api_key):
#     """
#     Utilise l'API Geocoding d'OpenWeatherMap
#     URL : http://api.openweathermap.org/geo/1.0/direct

#     Récupérez lat/lon pour chaque ville
#     """
#     pass

# définie au début
get_city_coordinates('montpellier')

{'name': 'Montpellier',
 'local_names': {'bg': 'Монпелие',
  'fr': 'Montpellier',
  'eo': 'Montpeliero',
  'oc': 'Montpelhièr',
  'lt': 'Monpeljė',
  'ar': 'مونبيلييه',
  'la': 'Monspessulanus',
  'ko': '몽펠리에',
  'uk': 'Монпельє',
  'ja': 'モンペリエ',
  'br': 'Moñtpelhier',
  'zh': '蒙彼利埃',
  'ca': 'Montpeller',
  'ru': 'Монпелье',
  'fa': 'مون\u200cپلیه',
  'sr': 'Монпеље',
  'he': 'מונפלייה',
  'el': 'Μονπελιέ',
  'mk': 'Монпелје'},
 'lat': 43.6112422,
 'lon': 3.8767337,
 'country': 'FR',
 'state': 'Occitania'}


## Partie 4 : Analyse et visualisation

### 📈 Analyses à réaliser

1. **Comparaison inter-villes**
   - Températures moyennes par ville
   - Variabilité climatique (écart-type)
   - Corrélations température/humidité

2. **Tendances temporelles**
   - Évolution sur 30 jours
   - Identification des patterns

3. **Qualité de l'air vs météo**
   - Impact de la pluie sur la pollution
   - Corrélations vent/qualité de l'air

In [51]:
### 💡 Visualisations guidées

# 1. Heatmap des températures par ville et jour
plt.figure(figsize=(15, 8))

# Créez un pivot table : villes en colonnes, dates en lignes
# Utilisez seaborn.heatmap()

# 2. Boxplot comparatif des précipitations
# Utilisez seaborn.boxplot()

# 3. Scatter plot qualité air vs température
# Ajoutez une regression line avec seaborn.regplot()



<Figure size 1500x800 with 0 Axes>

<Figure size 1500x800 with 0 Axes>

## Partie 5 : API bonus - Données agricoles

### 🌱 API AgroMonitoring (gratuite)

In [52]:
# API satellite pour l'agriculture
AGRO_API_KEY = "VOTRE_CLE_AGROMONITORING"

def get_soil_data(polygon_coordinates, api_key):
    """
    Récupère des données de sol via satellite
    URL : http://api.agromonitoring.com/agro/1.0/

    Données disponibles :
    - Indices de végétation (NDVI)
    - Humidité du sol
    - Température de surface
    """
    pass



**Défi avancé :** Créez des recommandations de plantation basées sur :
- Données météo des 30 derniers jours
- Prévisions à 5 jours
- Qualité de l'air
- Indices de végétation satellite



## 🏆 Livrables attendus

### 📊 Dashboard météo
Créez un tableau de bord contenant :
1. **Aperçu temps réel** des 6 villes
2. **Graphiques de tendances** sur 30 jours
3. **Alertes qualité de l'air** (AQI > 100)
4. **Recommandations agricoles** par ville

In [53]:
### 📱 Format de présentation

def generate_weather_report(city_data):
    """
    Génère un rapport automatisé

    Format :
    - Résumé exécutif (3 lignes)
    - Métriques clés (tableaux)
    - Graphiques (4 visualisations)
    - Recommandations (bullet points)
    """
    pass

---

## 🎓 Critères d'évaluation

- [ ] **APIs fonctionnelles** : Toutes les connexions API marchent
- [ ] **Gestion d'erreurs** : Code robuste avec try/except
- [ ] **Qualité des données** : Validation et nettoyage
- [ ] **Visualisations** : Graphiques informatifs et esthétiques
- [ ] **Insights business** : Recommandations basées sur les données

### 🔗 Préparation au Notebook 2
Le prochain notebook utilisera une vraie base de données PostgreSQL hébergée pour analyser des données de ventes e-commerce, en croisant avec vos données météo pour des analyses géolocalisées.

### 📚 APIs alternatives (si quotas dépassés)
- **WeatherAPI** : 1M appels/mois gratuits
- **AccuWeather** : 50 appels/jour gratuits  
- **Climatiq** : Données climat et carbone
- **NASA APIs** : Données satellite gratuites